In [1]:
# !pip install psycopg2
# !wget -P /content https://jdbc.postgresql.org/download/postgresql-42.2.27.jar
jdbc_driver_path = "/content/postgresql-42.2.27.jar"
import os
print(os.listdir())
# print(os.chdir())


['.config', 'postgresql-42.2.27.jar', '.ipynb_checkpoints', 'weather_20250224.json', 'sample_data']


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.jars", jdbc_driver_path) \
    .appName("WeatherAPIandETL") \
    .getOrCreate()



In [3]:
from datetime import datetime

print(datetime.now())
spark.sparkContext

2025-03-14 22:32:29.751992


<SparkContext master=local[*] appName=WeatherAPIandETL>

In [37]:
# from google.colab import files
# uploaded = files.upload()


In [38]:
# import os
# print(os.listdir())

['.config', 'weather_20250224.json', 'sample_data']


In [ ]:
# import json

# with open("/home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather.json", "r") as opfile:
#     data = json.load(opfile)

# print(data)

# with open("/home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather_new.json", "w") as f:
#     for record in data:
#         f.write(json.dumps(record) + "\n")  # ✅ Writes each object on a new line.


{'location': {'name': 'Piscataway', 'region': 'New Jersey', 'country': 'USA', 'xxx': 'aaa', 'lat': 40.5527, 'lon': -74.4582, 'tz_id': 'America/New_York', 'localtime_epoch': 1740332071, 'localtime': '2025-02-23 12:34'}, 'current': {'last_updated_epoch': 1740331800, 'last_updated': '2025-02-23 12:30', 'temp_c': 6.7, 'temp_f': 44.1, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 9.2, 'wind_kph': 14.8, 'wind_degree': 283, 'wind_dir': 'WNW', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 40, 'cloud': 0, 'feelslike_c': 3.9, 'feelslike_f': 39.0, 'windchill_c': 0.1, 'windchill_f': 32.3, 'heatindex_c': 3.3, 'heatindex_f': 38.0, 'dewpoint_c': -5.6, 'dewpoint_f': 21.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 3.2, 'gust_mph': 11.1, 'gust_kph': 17.9}, 'forecast': {'forecastday': [{'date': '2025-02-23', 'date_epoch': 1740268800, 'day': {'maxtemp_c': 8.3, 'maxtemp_f': 46.9, '

In [21]:
# df = spark.read.option("multiLine", True).json("/content/weather_20250224.json")

In [23]:
# df.printSchema()
# df.show()

root
 |-- current: struct (nullable = true)
 |    |-- cloud: long (nullable = true)
 |    |-- condition: struct (nullable = true)
 |    |    |-- code: long (nullable = true)
 |    |    |-- icon: string (nullable = true)
 |    |    |-- text: string (nullable = true)
 |    |-- dewpoint_c: double (nullable = true)
 |    |-- dewpoint_f: double (nullable = true)
 |    |-- feelslike_c: double (nullable = true)
 |    |-- feelslike_f: double (nullable = true)
 |    |-- gust_kph: double (nullable = true)
 |    |-- gust_mph: double (nullable = true)
 |    |-- heatindex_c: double (nullable = true)
 |    |-- heatindex_f: double (nullable = true)
 |    |-- humidity: long (nullable = true)
 |    |-- is_day: long (nullable = true)
 |    |-- last_updated: string (nullable = true)
 |    |-- last_updated_epoch: long (nullable = true)
 |    |-- precip_in: double (nullable = true)
 |    |-- precip_mm: double (nullable = true)
 |    |-- pressure_in: double (nullable = true)
 |    |-- pressure_mb: double (n

In [ ]:
# !ls -ltr /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather_deindent.json
#!chmod 777 /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather.json
#cat /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather.json | python -m json.tools

-rwxrwxrwx 1 tanvi tanvi 2971 Mar 12 19:29 /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather_deindent.json


In [24]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import *  # Optional: to use SQL functions like col(), lit(), etc.

# df.show()

# df.createOrReplaceTempView("weather_json")
# query = "SELECT * FROM weather_json"
# result = spark.sql(query)


+--------------------+--------------------+--------------------+
|             current|            forecast|            location|
+--------------------+--------------------+--------------------+
|{0, {1000, //cdn....|{[{{0, 0, 19, Wan...|{USA, 40.5527, 20...|
+--------------------+--------------------+--------------------+



In [25]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import get_json_object, col

# result = df.select( "location"
    # get_json_object("location", "$.name").alias("name")
    # )

# result.show(truncate=False)

# newdf = df.select("location", "current")

# result.show(5)

In [ ]:

# data = [
#     ("1", """{"name": "John", "age": 30, "address": {"city": "New York", "zip": "10001"}, "phones": [{"type": "home", "number": "212-555-1234"}, {"type": "work", "number": "646-555-5678"}]}"""),
#     ("2", """{"name": "Jane", "age": 25, "address": {"city": "Boston", "zip": "02108"}, "phones": [{"type": "mobile", "number": "617-555-4321"}]}""")
# ]

# samdf = spark.createDataFrame(data, ["id", "json_data"])

# # Extract various fields
# result = samdf.select(
#     "id",
#     get_json_object("json_data", "$.name").alias("name"),
#     get_json_object("json_data", "$.age").alias("age"),
#     get_json_object("json_data", "$.address.city").alias("city"),
#     get_json_object("json_data", "$.phones[0].number").alias("first_phone")
# )

# # result.show(truncate=False)
# samdf.show()

+---+--------------------+
| id|           json_data|
+---+--------------------+
|  1|{"name": "John", ...|
|  2|{"name": "Jane", ...|
+---+--------------------+



In [6]:
import requests
api_dict = {
"API_KEY": "c2fe5ebd74734989a33163320252302",
"BASE_URL": "http://api.weatherapi.com/v1/forecast.json",
}

params= {"key":api_dict["API_KEY"], "q": "08854", "days": "1", "aqi": "no", "alerts": "no"}
response = requests.get(api_dict["BASE_URL"], params=params)
weather_data = response.json()
print(weather_data)

{'location': {'name': 'Piscataway', 'region': 'New Jersey', 'country': 'USA', 'lat': 40.5514984130859, 'lon': -74.4589996337891, 'tz_id': 'America/New_York', 'localtime_epoch': 1741991645, 'localtime': '2025-03-14 18:34'}, 'current': {'last_updated_epoch': 1741991400, 'last_updated': '2025-03-14 18:30', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 120, 'wind_dir': 'ESE', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 60, 'cloud': 0, 'feelslike_c': 12.3, 'feelslike_f': 54.2, 'windchill_c': 8.3, 'windchill_f': 47.0, 'heatindex_c': 10.2, 'heatindex_f': 50.3, 'dewpoint_c': 7.2, 'dewpoint_f': 45.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.2, 'gust_mph': 9.8, 'gust_kph': 15.8}, 'forecast': {'forecastday': [{'date': '2025-03-14', 'date_epoch': 1741910400, 'day': {'maxtemp_c': 12.9, 'maxtemp_f': 

In [4]:
postgres_url = 'jdbc:postgresql://localhost:5432/postgres'

postgres_properties = {
    'user': 'postgres',
    'password': 'myPassword',
    'driver': 'org.postgresql.Driver'}


In [7]:
import uuid

single_uuid = uuid.uuid4()

In [8]:
from pyspark.sql.functions import lit
location_list = [weather_data['location']]
df_l = spark.createDataFrame(location_list)
df_l = df_l.withColumn("process_uuid", lit(str(single_uuid)))
df_l = df_l.withColumn("zipcode", lit("08854"))
df_l.show()

+-------+----------------+----------------+---------------+-----------------+----------+----------+----------------+--------------------+-------+
|country|             lat|       localtime|localtime_epoch|              lon|      name|    region|           tz_id|        process_uuid|zipcode|
+-------+----------------+----------------+---------------+-----------------+----------+----------+----------------+--------------------+-------+
|    USA|40.5514984130859|2025-03-14 18:34|     1741991645|-74.4589996337891|Piscataway|New Jersey|America/New_York|9529f561-7d21-4a1...|  08854|
+-------+----------------+----------------+---------------+-----------------+----------+----------+----------------+--------------------+-------+



In [9]:
df_l.select("name","region","country","lat","lon","tz_id","localtime_epoch","localtime","process_uuid","zipcode")
df_l.write.jdbc(
    url=postgres_url,
    table="stage.location",
    mode="overwrite",
    properties=postgres_properties
)

Py4JJavaError: An error occurred while calling o72.jdbc.
: org.postgresql.util.PSQLException: Connection to localhost:5432 refused. Check that the hostname and port are correct and that the postmaster is accepting TCP/IP connections.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:303)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:51)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:225)
	at org.postgresql.Driver.makeConnection(Driver.java:465)
	at org.postgresql.Driver.connect(Driver.java:264)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:161)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anon

In [43]:
currently_list = [weather_data['current']]
df_c = spark.createDataFrame(currently_list)
df_cf = df_c.selectExpr("cloud","condition.text as condition_text","condition.icon as condition_icon","condition.code as condition_code","dewpoint_c","dewpoint_f","feelslike_c","feelslike_f","gust_kph","gust_mph","heatindex_c","heatindex_f","humidity","is_day","last_updated","last_updated_epoch","precip_in","precip_mm","pressure_in","pressure_mb","temp_c","temp_f","uv","vis_km","vis_miles","wind_degree","wind_dir","wind_kph","wind_mph","windchill_c","windchill_f")
df_cf.show()

+-----+--------------+--------------------+--------------+----------+----------+-----------+-----------+--------+--------+-----------+-----------+--------+------+----------------+------------------+---------+---------+-----------+-----------+------+------+---+------+---------+-----------+--------+--------+--------+-----------+-----------+
|cloud|condition_text|      condition_icon|condition_code|dewpoint_c|dewpoint_f|feelslike_c|feelslike_f|gust_kph|gust_mph|heatindex_c|heatindex_f|humidity|is_day|    last_updated|last_updated_epoch|precip_in|precip_mm|pressure_in|pressure_mb|temp_c|temp_f| uv|vis_km|vis_miles|wind_degree|wind_dir|wind_kph|wind_mph|windchill_c|windchill_f|
+-----+--------------+--------------------+--------------+----------+----------+-----------+-----------+--------+--------+-----------+-----------+--------+------+----------------+------------------+---------+---------+-----------+-----------+------+------+---+------+---------+-----------+--------+--------+--------+--

In [44]:
daily_list = [weather_data['forecast']['forecastday'][0]['day']]
df_d = spark.createDataFrame(daily_list)
df_df = df_d.selectExpr("avghumidity","avgtemp_c","avgtemp_f","avgvis_km","avgvis_miles","condition.text as condition_text","condition.icon as condition_icon","condition.code as condition_code","daily_chance_of_rain","daily_chance_of_snow","daily_will_it_rain","daily_will_it_snow","maxtemp_c","maxtemp_f","maxwind_kph","maxwind_mph","mintemp_c","mintemp_f","totalprecip_in","totalprecip_mm","totalsnow_cm","uv")
df_df.show()

+-----------+---------+---------+---------+------------+--------------+--------------------+--------------+--------------------+--------------------+------------------+------------------+---------+---------+-----------+-----------+---------+---------+--------------+--------------+------------+---+
|avghumidity|avgtemp_c|avgtemp_f|avgvis_km|avgvis_miles|condition_text|      condition_icon|condition_code|daily_chance_of_rain|daily_chance_of_snow|daily_will_it_rain|daily_will_it_snow|maxtemp_c|maxtemp_f|maxwind_kph|maxwind_mph|mintemp_c|mintemp_f|totalprecip_in|totalprecip_mm|totalsnow_cm| uv|
+-----------+---------+---------+---------+------------+--------------+--------------------+--------------+--------------------+--------------------+------------------+------------------+---------+---------+-----------+-----------+---------+---------+--------------+--------------+------------+---+
|         76|      7.0|     44.5|     10.0|         6.0|       Cloudy |//cdn.weatherapi....|          1

In [51]:
hourly_list = weather_data['forecast']['forecastday'][0]['hour']
df_h = spark.createDataFrame(hourly_list)
df_hf = df_h.selectExpr("time_epoch","time","temp_c","temp_f","is_day","wind_mph","wind_kph","wind_degree","wind_dir","pressure_mb","pressure_in","precip_mm","precip_in","snow_cm","humidity","cloud","feelslike_c","feelslike_f","windchill_c","windchill_f","heatindex_c","heatindex_f","dewpoint_c","dewpoint_f","will_it_rain","chance_of_rain","will_it_snow","chance_of_snow","vis_km","vis_miles","gust_mph","gust_kph","uv","condition.text as condition_text","condition.icon as condition_icon","condition.code as condition_code")


PySparkTypeError: [CANNOT_MERGE_TYPE] Can not merge type `LongType` and `DoubleType`.